In [1]:
import spacy

from main import apps_config_factory
from text_to_command.configuration_units import SystemConfiguration, SessionConfiguration
from text_to_command.indexer import Indexer
from text_to_command.intent_resolver import IntentResolver

In [2]:
indexer = Indexer(spacy.load("en_core_web_md"))

In [8]:
# query = "Send Ann a message 'Buy some potato' with Telegram"
query = "Set sound to 50%"
# query = "Sound up"
# query = "Increase brightness on 10"  # problems!

intent_resolver = IntentResolver(indexer)
commands = intent_resolver.resolve_intent_recommendations(
    query,
    apps_config_factory(),
    SystemConfiguration([]), SessionConfiguration("", [])
)

for c in sorted(commands, key=lambda x: x.score, reverse=True)[:8]:
    print(round(c.score, 3), " | ", c)

0.838  |  SkillFunctionCommand(id=pc_system.Set sound level, distribution=f[0.8116221, 0.82641596, 0.6658929], params={'value': 50})
0.765  |  SkillFunctionCommand(id=pc_system.Sound up, distribution=f[0.6684831, 0.89088625, 0.7350132], params={'value': 50})
0.733  |  SkillFunctionCommand(id=pc_system.Sound down, distribution=f[0.6307921, 0.85796624, 0.71164715], params={'value': 50})
0.691  |  SkillFunctionCommand(id=pc_system.Set brightness level, distribution=f[0.6462962, 0.5294598, 0.49824846], params={'value': 50})
0.604  |  SkillFunctionCommand(id=pc_system.Mute sound, distribution=f[0.50266516, 0.7834334, 0.52677226], params={})
0.504  |  SkillFunctionCommand(id=alarm_clock.Start alarm clock, distribution=f[0.5218993, 0.51590365, 0.4750055], params={})
0.482  |  SkillFunctionCommand(id=pc_system.Brightness up, distribution=f[0.45371842, 0.48302445, 0.50855076], params={'value': 50})
0.469  |  SkillFunctionCommand(id=alarm_clock.Show nearest alarm, distribution=f[0.45261478, 0.46